<a href="https://colab.research.google.com/github/Vikk-17/college/blob/main/miniPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

file_path = "/content/expanded_balanced_traffic_popular_ports.csv"
df = pd.read_csv(file_path)
df.shape

(50000, 10)

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import joblib

# Global variables
scaler = None
label_encoders = {}

def preprocess_csv_with_dummies(file_path, is_training=True):
    """
    Preprocess CSV data with specific features
    """
    global scaler, label_encoders

    df = pd.read_csv(file_path)

    # Store original value distribution
    if is_training:
        categorical_distribution = {
            col: df[col].value_counts(normalize=True) for col in ['alert', 'classification', 'protocol']
        }
        joblib.dump(categorical_distribution, '/content/model/categorical_distribution.pkl')

    # Define numerical and categorical columns
    numerical_columns = ['src_port', 'dst_port', 'priority']
    categorical_columns = ['alert', 'classification', 'protocol']

    # Handle categorical columns with LabelEncoder
    for col in categorical_columns:
        if is_training:
            label_encoders[col] = LabelEncoder()
            df[col] = label_encoders[col].fit_transform(df[col])
        else:
            df[col] = label_encoders[col].transform(df[col])

    # Normalize numerical columns
    if is_training:
        scaler = MinMaxScaler()
        df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    else:
        df[numerical_columns] = scaler.transform(df[numerical_columns])

    # Store timestamp and IP addresses separately as they don't need preprocessing
    timestamp_data = df['timestamp']
    src_ip_data = df['src_ip']
    dst_ip_data = df['dst_ip']

    # Drop timestamp and IP columns for training
    df = df.drop(['timestamp', 'src_ip', 'dst_ip'], axis=1)

    return (df.astype(np.float32), numerical_columns, categorical_columns,
            timestamp_data, src_ip_data, dst_ip_data)


class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class VAE(tf.keras.Model):
    def __init__(self, original_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = self.build_encoder(original_dim, latent_dim)
        self.decoder = self.build_decoder(original_dim, latent_dim)
        self.latent_dim = latent_dim

    def build_encoder(self, original_dim, latent_dim):
        inputs = tf.keras.layers.Input(shape=(original_dim,))
        x = tf.keras.layers.Dense(256, activation="relu")(inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(128, activation="relu")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        z_mean = tf.keras.layers.Dense(latent_dim)(x)
        z_log_var = tf.keras.layers.Dense(latent_dim)(x)
        z = Sampling()([z_mean, z_log_var])
        return tf.keras.Model(inputs, [z_mean, z_log_var, z], name="encoder")

    def build_decoder(self, original_dim, latent_dim):
        latent_inputs = tf.keras.layers.Input(shape=(latent_dim,))
        x = tf.keras.layers.Dense(128, activation="relu")(latent_inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(256, activation="relu")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        outputs = tf.keras.layers.Dense(original_dim, activation="sigmoid")(x)
        return tf.keras.Model(latent_inputs, outputs, name="decoder")

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        reconstruction_loss = tf.keras.losses.mse(inputs, reconstructed)
        reconstruction_loss *= tf.cast(tf.shape(inputs)[1], tf.float32)
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        self.add_loss(tf.reduce_mean(reconstruction_loss + kl_loss))
        return reconstructed

class Generator(tf.keras.Model):
    def __init__(self, data_dim):
        super(Generator, self).__init__()
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(data_dim, activation='tanh')
        ])

    def call(self, inputs):
        return self.model(inputs)

class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Dense(1)
        ])

    def call(self, inputs):
        return self.model(inputs)

class Classifier(tf.keras.Model):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

    def call(self, inputs):
        return self.model(inputs)

def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

def gradient_penalty(discriminator, real_samples, fake_samples):
    alpha = tf.random.uniform([tf.shape(real_samples)[0], 1], 0.0, 1.0)
    interpolated = alpha * real_samples + (1 - alpha) * fake_samples

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        predictions = discriminator(interpolated)

    gradients = tape.gradient(predictions, interpolated)
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=1))
    gradient_penalty = tf.reduce_mean(tf.square(slopes - 1.0))
    return gradient_penalty

def train_wgan(generator, discriminator, data, latent_dim, batch_size=64, epochs=100, n_critic=5, gp_weight=10.0):
    gen_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.9)
    disc_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.9)

    for epoch in range(epochs):
        for _ in range(n_critic):
            # Train Discriminator
            with tf.GradientTape() as disc_tape:
                batch_indices = np.random.randint(0, data.shape[0], batch_size)
                real_data = data[batch_indices]

                z = tf.random.normal((batch_size, latent_dim))
                fake_data = generator(z, training=True)

                real_output = discriminator(real_data, training=True)
                fake_output = discriminator(fake_data, training=True)

                gp = gradient_penalty(discriminator, real_data, fake_data)
                disc_loss = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output) + gp_weight * gp

            grads_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
            disc_optimizer.apply_gradients(zip(grads_disc, discriminator.trainable_variables))

        # Train Generator
        with tf.GradientTape() as gen_tape:
            z = tf.random.normal((batch_size, latent_dim))
            fake_data = generator(z, training=True)
            fake_output = discriminator(fake_data, training=True)
            gen_loss = -tf.reduce_mean(fake_output)

        grads_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gen_optimizer.apply_gradients(zip(grads_gen, generator.trainable_variables))

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}")
            print(f"Generator Loss: {gen_loss:.4f}")
            print(f"Discriminator Loss: {disc_loss:.4f}")

def train_classifier(classifier, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )

    history = classifier.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping]
    )

    return history

def predict_traffic(classifier, data, numerical_columns, categorical_columns):
    """
    Predict traffic classification for new data
    """
    processed_data, _, _ = preprocess_csv_with_dummies(data, is_training=False)
    predictions = classifier.predict(processed_data)
    predicted_classes = np.argmax(predictions, axis=1)

    # Convert numeric predictions back to original labels
    original_labels = label_encoders['classification'].inverse_transform(predicted_classes)

    return original_labels

def denormalize_data(synthetic_data, original_df, numerical_columns, categorical_columns):
    """
    Denormalize synthetic data to match original format
    """
    # Load categorical distribution
    categorical_distribution = joblib.load('/content/model/categorical_distribution.pkl')

    # Convert to DataFrame
    df = pd.DataFrame(synthetic_data, columns=['alert', 'classification', 'priority','protocol', 'src_port', 'dst_port'])

    # Denormalize numerical columns
    if scaler:
        df[numerical_columns] = scaler.inverse_transform(df[numerical_columns])

        # Round ports and priority to integers and ensure valid ranges
        df['src_port'] = np.round(df['src_port']).astype(int).clip(1, 65535)
        df['dst_port'] = np.round(df['dst_port']).astype(int).clip(1, 65535)
        df['priority'] = np.round(df['priority']).astype(int).clip(1, 5)

    # Convert categorical columns back to original labels
    for col in categorical_columns:
        # predicted_classes = np.argmax(df[col].values.reshape(-1, 1), axis=1)
        # df[col] = label_encoders[col].inverse_transform(predicted_classes)
        possible_values = list(categorical_distribution[col].index)
        probabilites = list(categorical_distribution[col].values)

        # Generate values based on original distribution
        df[col] = np.random.choice(possible_values, size=len(df), p=probabilites)

    # Generate timestamps in the correct format
    base_timestamp = pd.Timestamp('2025-02-14')
    df['timestamp'] = [
        (base_timestamp + pd.Timedelta(seconds=i)).strftime('%m/%d-%H:%M:%S.%f')
        for i in range(len(df))
    ]


    # Generate diverse IP address
    def generate_realistic_ip():
        ranges = [
            ('192.168.', 16, 31),
            ('10.', 0, 255),
            ('172.', 16, 31),
            ('', 2, 255)
        ]
        range_choice = np.random.choice(len(ranges))
        prefix, start, end = ranges[range_choice]

        if prefix:
            return f"{prefix}{np.random.randint(start, end)}." + \
                   f"{np.random.randint(0, 256)}.{np.random.randint(1, 255)}"
        else:
            return f"{np.random.randint(start, end)}." + \
                   f"{np.random.randint(0, 256)}." + \
                   f"{np.random.randint(0, 256)}." + \
                   f"{np.random.randint(1, 255)}"
    df['src_ip'] = [generate_realistic_ip() for _ in range(len(df))]
    df['dst_ip'] = [generate_realistic_ip() for _ in range(len(df))]
    # Reorder columns to match original CSV
    df = df[['timestamp', 'alert', 'classification', 'priority', 'protocol', 'src_ip', 'src_port', 'dst_ip', 'dst_port']]

    return df


def validate_synthetic_data(original_df, synthetic_df):
    """
    Validate the diversity and distribution of synthetic data
    """
    print("\nData Distribution Analysis:")

    for col in original_df.columns:
        print(f"\n{col} distribution:")
        print("\nOriginal:")
        print(original_df[col].value_counts(normalize=True).head())
        print("\nSynthetic:")
        print(synthetic_df[col].value_counts(normalize=True).head())

        if col in ['src_port', 'dst_port', 'priority']:
            print(f"\n{col} statistics:")
            print("Original:", original_df[col].describe())
            print("Synthetic:", synthetic_df[col].describe())

def generate_synthetic_data(generator, num_samples, latent_dim, original_df):
    """
    Generate synthetic network traffic data
    """
    # Generate raw synthetic data
    z = tf.random.normal((num_samples, latent_dim))
    synthetic_data = generator(z, training=False).numpy()

    # Denormalize the data
    numerical_columns = ['src_port', 'dst_port', 'priority']
    categorical_columns = ['alert', 'classification', 'protocol']

    denormalized_data = denormalize_data(
        synthetic_data,
        original_df,
        numerical_columns,
        categorical_columns
    )

    return denormalized_data

def main():
    # Load and preprocess data
    file_path = '/content/mini.csv'
    original_df = pd.read_csv(file_path)

    # Process data
    (processed_data, numerical_columns, categorical_columns,
     timestamp_data, src_ip_data, dst_ip_data) = preprocess_csv_with_dummies(file_path)

    # Split data
    X = processed_data.values
    y = processed_data['classification'].values
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Initialize and train models
    original_dim = X.shape[1]
    latent_dim = 32
    num_classes = len(np.unique(y))

    # Train VAE
    vae = VAE(original_dim, latent_dim)
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    vae.fit(X_train, X_train, validation_data=(X_val, X_val), epochs=50, batch_size=64)

    # Train WGAN
    generator = Generator(original_dim)
    discriminator = Discriminator()
    train_wgan(generator, discriminator, X_train, latent_dim)

    # # Train Classifier
    # classifier = Classifier(original_dim, num_classes)
    # history = train_classifier(classifier, X_train, y_train, X_val, y_val)

    # # Evaluate classifier
    # test_loss, test_accuracy = classifier.evaluate(X_test, y_test)
    # print(f"Test accuracy: {test_accuracy:.4f}")

    # Save models and preprocessing objects
    tf.keras.models.save_model(classifier, '/content/model/classifier_model.keras')
    joblib.dump(scaler, '/content/model/scaler.pkl')
    joblib.dump(label_encoders, '/content/model/label_encoders.pkl')

    # Generate synthetic data
    num_samples = 1000
    synthetic_df = generate_synthetic_data(generator, num_samples, latent_dim, original_df)

    # Save synthetic data
    synthetic_df.to_csv('/content/synthetic_mini.csv', index=False)

    validate_synthetic_data(original_df, synthetic_df)

    # Verify data structure
    print("\nColumn names match:",
          all(synthetic_df.columns == original_df.columns))
    print("\nData types:")
    for col in synthetic_df.columns:
        print(f"{col}: {synthetic_df[col].dtype}")

if __name__ == "__main__":
    main()


Epoch 1/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 14.0358 - val_loss: 13.9603
Epoch 2/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 13.7588 - val_loss: 13.9630
Epoch 3/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 13.8254 - val_loss: 13.9609
Epoch 4/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 13.7596 - val_loss: 13.9631
Epoch 5/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 13.7859 - val_loss: 13.9582
Epoch 6/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 13.8006 - val_loss: 13.9596
Epoch 7/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 13.7906 - val_loss: 13.9577
Epoch 8/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 13.7048 - val_loss: 13.9564
Epoch 9/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 13.6429 - val_loss: 13.9585
Epoch 10/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 13.8087 - val_loss: 13.9565
Epoch 11/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 13.8490 - val_loss: 13.9561
Epoch 12/50
547/547 ━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1094/1094 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9831 - loss: 0.0590 - val_accuracy: 1.0000 - val_loss: 4.7448e-07
Epoch 2/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 1.0000 - loss: 2.8709e-07 - val_accuracy: 1.0000 - val_loss: 7.1446e-08
Epoch 3/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 1.0000 - loss: 5.3176e-08 - val_accuracy: 1.0000 - val_loss: 2.2475e-08
Epoch 4/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 1.0000 - loss: 1.8066e-08 - val_accuracy: 1.0000 - val_loss: 9.9818e-09
Epoch 5/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 1.0000 - loss: 7.1602e-09 - val_accuracy: 1.0000 - val_loss: 4.7048e-09
Epoch 6/50
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 1.0000 - loss: 3.6252e-09 - val_accuracy: 1.0000 - val_loss: 2.3842e-09
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 5.1685e-07
Test accuracy: 1.0000

Data Distribution Analysis:

timestamp distribution:

Original:
timestamp
02/14-00:12